## Grouping

The pandas `groupby` function allows us to group our data on the values in a column or column to look at summary measures for records sharing the same values.

For example, let's load the speed camera dataset again and ask which camera locations or days of the week have produced the most violations.

In [ ]:
import pandas as pd
df = pd.read_csv("data/Speed_Camera_Violations.csv")
df.head()

Now that the data is loaded, let's find the 10 locations with the most total violations recorded.

To do this, we need to group by the ADDRESS column, then examine the VIOLATIONS column of the resulting grouped dataframe.

In [ ]:
# first let's group by address and look at descriptive statistics for the first 10 records
df.groupby(["ADDRESS"])["VIOLATIONS"].describe().head(10)

The above records aren't sorted in any meaningful way, but the first thing to note is that the Index is no longer just an integer, it is now the Address. This is because the `groupby` method returns a special object with a new index made up of the 
values of the column being grouped on.

We can still use the `loc` indexer with this new grouped object to, for example, find the count for a given address:

In [ ]:
# `count` returns the number of rows for this address, not the total violation count.
# IE this tells us the number of observation (in case of our example data, Speed_Camera_Violations,
# this corresponds to the number of different days with at least one violation).
df.groupby(["ADDRESS"])["VIOLATIONS"].count().loc["19 W CHICAGO AVE"]

In [ ]:
# to get the total violation count, we want the `sum` method:
df.groupby(["ADDRESS"])["VIOLATIONS"].sum().loc["19 W CHICAGO AVE"]

In [ ]:
# Now let's get the top 10 camera locations by total violation count:
df.groupby(["ADDRESS"])["VIOLATIONS"].sum().sort_values(ascending=False).head(10)

It's possible that some locations just have more observations than others, so a more meaningful measure is probably the mean violation count per observation. To get this we just need to use the `mean` function rather than `sum`.

In [ ]:
df.groupby(["ADDRESS"])["VIOLATIONS"].mean().sort_values(ascending=False).head(10)

How about days of the week? *When* are people most likely to be caught speeding?

The simplest way to do this is to create a new weekday column and group on that.

In [ ]:
# datetime series have a special `dt` property that exposes the date/time-specific functionality.
# In this case, dayofweek is a 0-based index where 0 = Monday, 6 = Sunday.
df["VIOLATION DATE"] = pd.to_datetime(df["VIOLATION DATE"], format="%m/%d/%Y")
df["VIOLATION DATE"].dt.dayofweek.head()

In [ ]:
df["DAY OF WEEK"] = df["VIOLATION DATE"].dt.dayofweek
df.groupby(["DAY OF WEEK"])["VIOLATIONS"].mean()

## Plotting

It's not easy to understand at a glance the distribution of speeding violations by day of the week above, so let's produce a simple plot to visualize and help understand it.

Pandas has some basic plotting functions, but I prefer how it interacts with a different visualization package called Seaborn

If you do not have seaborn, you can use pip to install it pip install seaborn

In [ ]:
import seaborn as sns #The cannonical way to import seaborn

In [ ]:
# The beauty of pandas with seaborn is how cleanly they interact with eachother

sns.lineplot(x = 'DAY OF WEEK', y = 'VIOLATIONS', data = df)


What if time of year is a factor here? Seaborn has a wonderful feature called hue, which allows for a quick comparison of different types of data in one graph

In [ ]:
#first lets create a month column
df['MONTH'] = df['VIOLATION DATE'].dt.month

#then lets recreate that same plot but with the months separated out
sns.lineplot(x = 'DAY OF WEEK', y = 'VIOLATIONS', hue = 'MONTH', data = df)

In [ ]:
#thats a little chaotic, lets try flipping the hue with the x axis

sns.lineplot(x = 'MONTH', y = 'VIOLATIONS', hue = 'DAY OF WEEK', data = df)

We can continue to make this graph prettier, but I will save that for the data visualization course, this is enough to see 2 interesting trents.  First is that violations are much more likely to occur on saturday and sunday, regardless of the time of year.  The second is that that violations are much more likely to occur during the summer months.

## Combining DataFrames

Often you will need to combine data from multiple data sets together. There are three types of combinations in pandas: concatenations and merges (aka joins).

**Concatenating** means taking multiple DataFrame objects and appending their rows together to make a new DataFrame. In general you will do this when your datasets contain the same columns and you are combining observations of the same type together into one dataset that contains all the rows from all the datasets.

**Merging** is joining DataFrames together SQL-style by using common values. This is useful when you have multiple datasets with common keys and you want to combine them into one dataset that contains columns from all the datasets being merged.

In [ ]:
# Concatenation example
df1 = pd.DataFrame({'Site': [1, 2, 3],
                    'Observed Value': [8.1, 5.5, 6.9]})

df2 = pd.DataFrame({'Site': [7, 8, 9],
                    'Observed Value': [10.5, 11.5, 12.0]})

print("df1: ")
print(df1)
print()
print("df2: ")
print(df2)
print()
print("concatenated along rows: ")
print(pd.concat([df1, df2]))
print()
print("concatenated along columns: ")
print(pd.concat([df1, df2], axis = 1))


In [ ]:
# Merge example
df1 = pd.DataFrame({'Site': [3, 1, 2],
                    'Observed Value': [8.1, 5.5, 6.9]})

df2 = pd.DataFrame({'Site': [1, 2, 3, 4],
                    'Temperature': [27.1, 18.2, 29.8, 30.4]})

print("df1: ")
print(df1)
print()
print("df2: ")
print(df2)
print()
print("merged: ")
print(pd.merge(df1, df2))

In [ ]:
print("df1: ")
print(df1)
print()
print("df2: ")
print(df2)
print()
print("merged: ")
print(pd.merge(df1, df2, how = 'outer'))

## Mapping and Applying

As we have already seen, there are some basic ways to create a new column based on existing columns.  But what if we have a more complicated function? For that, pandas provides the `map` and the `apply` functions

In [ ]:
#Mapping is commonly used to apply a dictionary of values to a column. 
#for instance lets take our lakes dataframe from before


lakes_data = [['erie', 64, 19],
            ['huron', 229, 59],
            ['michigan', 281, 85],
            ['ontario', 244, 86],
            ['superior', 406, 149]]

lakes = pd.DataFrame(lakes_data, columns = ['lake', 'Max Depth (m)', 'Avg Depth (m)'])
lakes

In [ ]:
#lets say I want to shorten the lake names

short_lake_dict = {'erie': 'ER',
                  'huron': 'HUR',
                  'michigan': 'MIC',
                  'ontario': 'ONT',
                  'superior': 'SUP'}

lakes.lake = lakes.lake.map(short_lake_dict)
lakes

But what if you want to do something more complicated than a dictionary? that is where the `apply` function comes into play.

In [ ]:
#first lets create a fake function to apply to our dataframe
#lets say on some lakes we care about the average depth, while for others we care about max depth 
def lake_specific_stats(lake, m, a):
    if lake in ['ER', 'MIC']:
        return m
    else:
        return a
    
#we will want to make a new column that captures the data (average or max depth) that we care about

lakes['max/ave'] = lakes.apply(lambda x: lake_specific_stats(x.lake, x['Max Depth (m)'], x['Avg Depth (m)']), axis = 1)
lakes
